In [30]:
import pkuseg
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

seg = pkuseg.pkuseg()

In [31]:
article_names = os.listdir('data')

In [32]:
article_contents = list()
for article_name in article_names:
    file_name = 'data/' + article_name
    file = open(file_name, 'r')
    article_content = file.read()
    file.close()
    article_contents.append(article_content)

In [33]:
article_words = list()
for i in tqdm(range(len(article_contents))):
    article_content = article_contents[i]
    words = seg.cut(article_content)
    article_words.append(words)

100%|██████████| 73/73 [00:18<00:00,  3.93it/s]


In [34]:
def counter(l: list) -> dict:
    stats = dict()
    for i in range(len(l)):
        item = l[i]
        if item not in stats:
            stats[item] = 1
        else:
            stats[item] = stats[item] + 1
    
    return stats

In [35]:
article_stats = list()
for i in tqdm(range(len(article_words))):
    article_stats.append(counter(article_words[i]))

100%|██████████| 73/73 [00:00<00:00, 714.13it/s]


In [36]:
all_term_stats = dict()
for i in tqdm(range(len(article_stats))):
    article_stat = article_stats[i]
    for word, count in article_stat.items():
        if word not in all_term_stats:
            all_term_stats[word] = count
        else:
            all_term_stats[word] = all_term_stats[word] + count

100%|██████████| 73/73 [00:00<00:00, 2145.72it/s]


In [37]:
all_term_vector = list(all_term_stats.keys())

In [38]:
doc_matrix = list()
for i in tqdm(range(len(article_stats))):
    stat_of_this_article = article_stats[i]
    doc_matrix_row = list()
    for term in all_term_vector:
        if term in stat_of_this_article:
            count = stat_of_this_article[term]
        else:
            count = 0
        doc_matrix_row.append(count)
    
    doc_matrix.append(doc_matrix_row)

100%|██████████| 73/73 [00:00<00:00, 180.62it/s]


In [39]:
len(doc_matrix)

73

In [40]:
len(doc_matrix[0])

14992

In [41]:
article_indexes = pd.DataFrame({
    'article_name': article_names,
    'row_num_in_doc_matrix': range(len(article_names))
})

In [42]:
term_indexes = pd.DataFrame({
    'term': all_term_vector,
    'col_num_in_doc_matrix': range(len(all_term_vector))
})

In [43]:
article_indexes.to_csv('termdocmatrix/article_indexes.csv', index=False)

In [44]:
term_indexes.to_csv('termdocmatrix/term_indexes.csv', index=False)

In [45]:
np.savetxt(
    fname = 'termdocmatrix/doc_matrix.txt',
    X = np.array(doc_matrix).astype(int),
    delimiter = ',',
    newline = '\n',
    fmt = '%u'
)